## Chapter 7. First-Class Functions

Functions in Python are first-class objects. Programming language researchers define a “first-class object” as a program entity that can be:
- Created at runtime
- Assigned to a variable or element in a data structure
- Passed as an argument to a function
- Returned as the result of a function

Integers, strings, and dictionaries are other examples of first-class objects in Python—nothing fancy here.

### Treating a Function Like an Object

In [1]:
# Example 7-1. Create and test a function, then read its __doc__ and check its type

def factorial(n):
    '''returns n!'''
    return 1 if n < 2 else n * factorial(n-1)

print(factorial(42))
print(factorial.__doc__)
print(type(factorial))

1405006117752879898543142606244511569936384000000000
returns n!
<class 'function'>


In [3]:
# Example 7-2. Use function through a different name, and pass function as argument

fact = factorial
print(fact)
print(fact(5))

print(map(factorial, range(11)))
print(list(map(factorial, range(11))))

<function factorial at 0x000001DDD8EBFD00>
120
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


### Higher-Order Function

A function that takes a function as an argument or returns a function as the result is a
higher-order function.

In [8]:
# Example 7-3. Sorting a list of words by length

fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)  # Higher-Order Function: it takes len() as an argument

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [9]:
# Example 7-4. Sorting a list of words by their reversed spelling

def reverse(word):
    return word[::-1]

print(reverse('testing'))
print(sorted(fruits, key=reverse))

gnitset
['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']


#### Modern Replacements for map, filter, and reduce

In [4]:
# Example 7-5. Lists of factorials produced with map and filter compared to alternatives coded as list comprehensions

print(list(map(fact, range(6))))
print([fact(n) for n in range(6)])  # a list comprehension

print(list(map(factorial, filter(lambda n: n % 2, range(6)))))
print([factorial(n) for n in range(6) if n % 2])  # List comprehension without map, filter and lambda

[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]
[1, 6, 120]
[1, 6, 120]


In [5]:
# Example 7-6. Sum of integers up to 99 performed with reduce and sum

from functools import reduce
from operator import add

print(reduce(add, range(100)))
print(sum(range(100)))  # Same task using sum; without importing 

4950
4950


In [2]:
# Extra: Python all() and any() builtins
# ref: https://docs.python.org/3/library/functions.html#all

# Return True if all elements of the iterable are true (or if the iterable is empty).
# Equivalent to:
def all(iterable):
    for element in iterable:
        if not element:
            return False
    return True

# Return True if any element of the iterable is true. If the iterable is empty, return False.
# Equivalent to:
def any(iterable):
    for element in iterable:
        if element:
            return True
    return False

In [22]:
# Example 7-7. Sorting a list of words by their reversed spelling using lambda

fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## The Nine Flavors of Callable Objects

In [23]:
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

### User-Defined Callable Types

- User-defined functions
    - Created with def statements or lambda expressions.
- Built-in functions
    - A function implemented in C (for CPython), like len or time.strftime.
- Built-in methods
    - Methods implemented in C, like dict.get.
- Methods
    - Functions defined in the body of a class.
- Classes
    - When invoked, a class runs its __new__ method to create an instance, then __init__ to initialize it, and finally the instance is returned to the caller. Because there is no new operator in Python, calling a class is like calling a function.2
- Class instances
    - If a class defines a __call__ method, then its instances may be invoked as functions— that’s the subject of the next section.
- Generator functions
    - Functions or methods that use the yield keyword in their body. When called, they return a generator object.
- Native coroutine functions
    - Functions or methods defined with async def. When called, they return a coroutine object. Added in Python 3.5.
- Asynchronous generator functions
    - Functions or methods defined with async def that have yield in their body. When called, they return an asynchronous generator for use with async for. Added in Python 3.6.

In [3]:
# Example 7-8. bingocall.py: A BingoCage does one thing: picks items from a shuffled list

import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        return self.pick()

In [4]:
bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())  # call __call__
print(callable(bingo))

2
1
True


### Function Introspection

In [26]:
dir(factorial)  # return the list of attributes

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [15]:
# Listing attributes of functions that don’t exist in plain instances

class C: pass
obj = C()
def func(): pass
# list attributes of functions that don't exist in plain instances
sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

### From Positional to Kerywork-Only Parameters

In [16]:
# Example 7-9. tag generates HTML; a keyword-only argument cls is used to pass
# “class” attributes as a workaround because class is a keyword in Python

def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [17]:
# Example 7-10. Some of the many ways of calling the tag function from Example 7-9

print(tag('br'))
print(tag('p', 'hello'))
print(tag('p', 'hello', 'world'))
print(tag('p', 'hello', id=33))

print(tag('p', 'hello', 'world', cls='sidebar'))
print(tag(content='testing', name="img"))

my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)  # ** passes all its items as separate arguments

<br />
<p>hello</p>
<p>hello</p>
<p>world</p>
<p id="33">hello</p>
<p class="sidebar">hello</p>
<p class="sidebar">world</p>
<img content="testing" />


'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

In [3]:
def f(a, *, b):  # *, b is keyword-only argument
    return a, b

print(f(1, b=2))
# print(f(1, 2, 3))  # Type error 

(1, 2)


In [20]:
def divmod(a, b, /):  # positional-only argument
    return (a // b, a % b)

print(divmod(20, 5))
# print(divmod(a=10, b=5))  # error

(4, 0)


### Packages for Functional Programming
#### The Operator Module

In [21]:
# Example 7-11. Factorial implemented with reduce and an anonymous function

from functools import reduce
from operator import mul

def factorial_reduce_lambda(n):
    return reduce(lambda a, b: a * b, range(1, n+1))

def factorial_reduce_mul(n):
    return reduce(mul, range(1, n+1))

print(factorial_reduce_lambda(4))
print(factorial_reduce_lambda(5))
print(factorial_reduce_mul(4))
print(factorial_reduce_mul(5))

24
120
24
120


In [22]:
# Example 7-13. Demo of itemgetter to sort a list of tuples (data from Example 2-8)

metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
    ]

from operator import itemgetter

for city in sorted(metro_data, key = itemgetter(1)):  # itemgetter replaces the below lambda
#for city in sorted(metro_data, key = lambda fields: fields[1]):
    print(city)
print()

cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


In [54]:
# Example 7-14. Demo of attrgetter to process a previously defined list of namedtuplecalled metro_data

from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
    for name, cc, pop, (lat, long) in metro_data]

print(metro_areas[0])
print(metro_areas[0].coord.lat)
print()

from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))
35.689722

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [56]:
import operator
[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

In [57]:
# Example 7-15. Demo of methodcaller: second test shows the binding of extra arguments

from operator import methodcaller

s = 'The time has come'
upcase = methodcaller('upper')
print(upcase(s))
hiphenate = methodcaller('replace', ' ', '-')
print(hiphenate(s))

THE TIME HAS COME
The-time-has-come


#### Freezing Arguments with functools.partial

In [23]:
# Example 7-16. Using partial to use a two-argument function where a one-argument callable is required

from operator import mul
from functools import partial

triple = partial(mul, 3)  # binding first positional argument to 3
print(triple(7))

list(map(triple, range(1,10)))

21


[3, 6, 9, 12, 15, 18, 21, 24, 27]

In [59]:
# Example 7-17. Building a convenient Unicode normalizing function with partial

import unicodedata, functools
nfc = functools.partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
print(s1, s2)
print(s1 == s2)
print(nfc(s1) == nfc(s2))

café café
False
True


In [64]:
# Example 7-18. Demo of partial applied to the function tag from Example 7-9
print(tag)
print()

from functools import partial
picture = partial(tag, 'img', cls='pic-frame')
print(picture(src='wumpus.jpeg'))
print(picture)
print(picture.func)
print(picture.args)
print(picture.keywords)

<function tag at 0x00000224596452D0>

<img class="pic-frame" src="wumpus.jpeg" />
functools.partial(<function tag at 0x00000224596452D0>, 'img', cls='pic-frame')
<function tag at 0x00000224596452D0>
('img',)
{'cls': 'pic-frame'}
